In [1]:
import qiskit
import numpy as np

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal.jigsaw import jigsaw
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear


from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar

from PatchedMeasCal.fake_backends import Grid, Hexagonal16, FullyConnected

from PatchedMeasCal.state_prep_circuits import GHZ_prep, GHZ_state_dist


from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial
import mthree
import qiskit.tools.jupyter

qiskit.IBMQ.load_account()
provider = qiskit.IBMQ.get_provider()

%qiskit_job_watcher

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

## Setup
The idea here is that each approachs gets a maximum of 32000 measurement shots to spend
- Backend style methods will spend 50% of that on the build, 50% of that on the run
- Circuit style methods will spend 50% on their full run, and the other 50% on other circuits that need to be exectued
- AIM will divide theirs up evenly

In [2]:
backend = provider.get_backend('ibmq_manila')

n_qubits = len(backend.properties().qubits)
n_meas_shots = 16000
n_build_shots = 16000
n_shots_cmc = n_build_shots // (2 * len(backend.configuration().coupling_map))


n_circuit_method_shots = n_meas_shots + n_build_shots

circuit = GHZ_prep(backend)

err_cmap = [[1, 3], [1, 4], [1, 2], [3, 4], [0, 1]]

## Qiskit Full Build

In [ ]:
n_shots_qiskit_full = n_build_shots // (2 ** n_qubits) 

full_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_full)

# n Circuits to execute
n_shots_qiskit_partial = n_build_shots // (n_qubits)

linear_filter = qiskit_full(backend, n_qubits, n_shots_qiskit_partial)


## Run with repetitions
If everything above is working, let's crank it out a few times

In [ ]:
n_reps = 10
results = {
    'bare':[],
    'full':[],
    'linear':[],
    'aim':[],
    'sim':[],
    'jigsaw':[],
    'cmc':[],
    'cmc_err':[]
}

for _ in range(n_reps):

    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    
    tpf_err = TensorPatchFitter(backend, n_shots=n_shots_cmc, coupling_map=err_cmap)
    tpf_err.build(verbose=True)
    
    tpf = TensorPatchFitter(backend, n_shots=n_shots_cmc)
    tpf.build(verbose=True)
    
    bare_result = bare_result_job.result().get_counts()
    
    results['bare'].append(
        GHZ_state_dist(bare_result)
    )
    results['full'].append(
        GHZ_state_dist(full_filter.apply(bare_result))
    )
    results['linear'].append(
            GHZ_state_dist(linear_filter.apply(bare_result))
    )
    results['cmc'].append(
            GHZ_state_dist(tpf.apply(bare_result))
    )
    results['cmc_err'].append(
            GHZ_state_dist(tpf_err.apply(bare_result))
    )
    results['aim'].append(GHZ_state_dist(aim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['sim'].append(GHZ_state_dist(sim(circuit, backend, n_qubits, n_shots=n_circuit_method_shots, equal_shot_distribution=True)))
    results['jigsaw'].append(GHZ_state_dist(jigsaw(circuit, backend, n_circuit_method_shots, equal_shot_distribution=True)))



In [6]:
mit = mthree.M3Mitigation(backend)
mit.cals_from_system(list(range(n_qubits)), n_build_shots // 2)

ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f9e74dc3af0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f9e74dc3af0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f9e74dc3af0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websocket:error from callback <bound method BaseWebsocketClient.on_close of <qiskit.providers.ibmq.api.clients.websocket.WebsocketClient object at 0x7f9e74dc3af0>>: on_close() missing 2 required positional arguments: 'status_code' and 'msg'
ERROR:websoc

In [7]:
dist_measure = GHZ_state_dist
n_reps = 10
for i in range(n_reps):
    bare_result_job = qiskit.execute(circuit, 
                         backend, 
                         shots=n_meas_shots, 
                         optimization_level=0,
                         initial_layout=list(range(n_qubits))
                        )
    bare_result = bare_result_job.result().get_counts()
    print(dist_measure(mit.apply_correction(bare_result, list(range(n_qubits))).nearest_probability_distribution()))

ERROR:websocket:close status: 4002


0.029499101072440137


ERROR:websocket:close status: 4002


0.026144572067201832


ERROR:websocket:close status: 4002


...

0.015050916149796423


ERROR:websocket:close status: 4002


0.10766143670214501


ERROR:websocket:close status: 4002


0.023297269025060574


In [ ]:
results

In [ ]:
ghz_manila_results = {'bare': [0.2963125,
  0.17931249999999999,
  0.17312499999999997,
  0.173875,
  0.15606250000000005],
 'full': [0.20368415681045032,
  0.06906851608570297,
  0.08437502800706814,
  0.06715740226609074,
  0.05554795126652545],
 'linear': [0.13761912336161208,
  0.10192057905096719,
  0.12468340526996569,
  0.10607253314285875,
  0.09467239058780025],
 'aim': [0.2063333333333333,
  0.17141666666666666,
  0.186,
  0.15633333333333332,
  0.15504166666666663],
 'sim': [0.187625,
  0.17912500000000003,
  0.2210625,
  0.16778125,
  0.18993749999999998],
 'jigsaw': [0.15333786356040274,
  0.11545855466820476,
  0.23779651621328113,
  0.2039240058459163,
  0.17408315492421728],
 'cmc': [0.22262870237898374,
  0.1241559970874444,
  0.1332083571273437,
  0.08498339218802736,
  0.11248285163839217],
 'cmc_err': [0.211963365840158,
  0.11202063049277111,
  0.12778236918119468,
  0.10605238073761358,
  0.10688117019739551]}

In [ ]:
res = ghz_manila_results
for r in res:
    avg = np.mean(res[r])
    h_bound = np.max(res[r]) - avg
    l_bound = avg - np.min(res[r])
    print(r, " & ", "$", "%.2f" % avg, "\substack{+", "%.2f" % h_bound, " \\\\ -", "%.2f" %l_bound, "}$", sep='')